In [9]:
import torch
import pandas as pd
import json
import os
import sys

scripts_dir = os.getcwd()
# Go up one level
project_root = os.path.abspath(os.path.join(scripts_dir, '..'))
sys.path.append(project_root)

from src.CNN import BeamformingCNN

In [8]:
c = BeamformingCNN(3, 5, 30)